In [ ]:
#@title upload an image
from google.colab import files
from PIL import Image as Impil
import numpy as np
from IPython.display import Image
import matplotlib.pyplot as plt
from matplotlib import colors
import math as mt
from numpy import linalg as LA
from mpl_toolkits.mplot3d import Axes3D
from sklearn.datasets import fetch_olivetti_faces
from sympy import *
init_printing(use_unicode=True)
%matplotlib inline

uploaded = files.upload()
print('------------------------------------------------------------------------')
print()
img = Impil.open(list(uploaded)[0])
imgGray = img.convert('L')
expr = str(list(uploaded)[0])
imgGray.save(expr[:expr.index('.')]+'_grayscale.jpg')
M0 = np.array(imgGray)

display(imgGray)
print('------------------------------------------------------------------------')
print()
print('Matrix of the image')
print('size =',np.shape(M0))
print('total number of pixels = ',np.shape(M0)[0]*np.shape(M0)[1])
print()
print(M0)


In [ ]:
#@title Matrix
import sympy as sp
from IPython.display import HTML, Math, display
#display(Math(sp.latex(sp.Matrix(M0))))

In [ ]:
#@title Compression of an image using first $k$ singular values
k = 40 #@param {type:"slider", min:1, max:200, step:1}

# Reading the image
M = plt.imread("/content/"+expr[:expr.index('.')]+'_grayscale.jpg')
# SVD
U, s, VT = LA.svd(M, full_matrices=True)
Sigma = np.zeros((M.shape[0], M.shape[1]))
Sigma[:min(M.shape[0], M.shape[1]), :min(M.shape[0], M.shape[1])] = np.diag(s)

if (True):
    M1 = U[:, :k] @ Sigma[:k, :k] @ VT[:k, :]
    max1=M1.max()
    for i in range (len(M1)):
        for j in range(len(M1[0])):
            M1[i][j]=round(M1[i][j]*(255/max1),0)

    img_approx = Impil.fromarray(M1)
    img_approx = img_approx.convert("L")
    expr = str(list(uploaded)[0])
    img_approx.save(expr[:expr.index('.')]+'_first_k_PCs.jpg')
    #img_approx.show()
    display(img_approx)

    print('------------------------------------------------------------------------')
    print()
    print('Matrix of the Compressed image')
    print('size of U =',np.shape(U[:, :k]))
    print('size of Sigma =',np.shape(Sigma[:k, :k]))
    print('size of V transpose =',np.shape(VT[:k, :]))
    print('total number of pixels =',np.shape(U[:, :k])[0]*np.shape(U[:, :k])[1] + np.shape(Sigma[:k, :k])[0]*np.shape(Sigma[:k, :k])[1] + np.shape(VT[:k, :])[0]*np.shape(VT[:k, :])[1])
    print()
    print(M1)


In [ ]:
#@title Compression of an image for remaining $n-k$ singular values
#k = 1 #@param {type:"slider", min:0, max:100, step:1}

if (True):
    M2 = U[:, k:] @ Sigma[k:, k:] @ VT[k:, :]
    max2=M2.max()
    min2=M2.min()
    diff2=max2 - min2
    for i in range (len(M2)):
        for j in range(len(M2[0])):
            M2[i][j]=(round((M2[i][j]-(min2))*(255/diff2),0))
            #M2[i][j]=round(M2[i][j],0)

    img_approx = Impil.fromarray(M2)
    img_approx = img_approx.convert("L")
    expr = str(list(uploaded)[0])
    img_approx.save(expr[:expr.index('.')]+'_remain_n-k_PCs.jpg')
    #img_approx.show()
    display(img_approx)

    print('------------------------------------------------------------------------')
    print()
    print('Matrix of the Compressed image')
    print('size of U =',np.shape(U[:, k:]))
    print('size of Sigma =',np.shape(Sigma[k:, k:]))
    print('size of V transpose =',np.shape(VT[k:, :]))
    print()
    print(M2)


In [ ]:
#@title Compression of an image using singular values between $k_1$ to $k_2$ { form-width: "20%" }
k1 = 41 #@param {type:"slider", min:0, max:100, step:1}
k2 = 100 #@param {type:"slider", min:0, max:100, step:1}

if(k2 > k1):
    k=int(k2-k1)

    M3 = U[:, k1:k2] @ Sigma[k1:k2, k1:k2] @ VT[k1:k2, :]

    max3 = M3.max()
    min3 = M3.min()
    diff3 = max3 - min3

    for i in range (len(M2)):
        for j in range(len(M2[0])):
            M3[i][j]=(round((M3[i][j]-(min3))*(255/diff3),0))

    img_approx = Impil.fromarray(M3)
    img_approx = img_approx.convert("L")
    expr = str(list(uploaded)[0])
    img_approx.save(expr[:expr.index('.')]+'_k1_to_k2_PCs.jpg')
    #img_approx.show()
    display(img_approx)

    print('------------------------------------------------------------------------')
    print()
    print('Matrix of the Compressed image')
    print('size of U =',np.shape(U[:, k1:k2]))
    print('size of Sigma =',np.shape(Sigma[k1:k2, k1:k2]))
    print('size of V transpose =',np.shape(VT[k1:k2, :]))
    print()
    print(M3)

else:
    print('Kindly select k2 > k1')

In [ ]:
#@title Image resize (by factor of float)
f1 = 2 #@param {type:"slider", min:1, max:10, step:0.1}
f = int(f1)
r = f1 - f
m = M.shape[0]
n = M.shape[1]
m1 = m//f
n1 = n//f

# M_resize if generated from M by adding entries
M_resize = np.zeros((m1, n1))

i1 = 0
i = 0
while(i1 < m1):
    j = 0
    j1 = 0
    while(j1 < n1):
        try:
            av = M[i][j]
            M_resize[i1][j1]=av
        except:
            pass
        j=j+f
        j1=j1+1

    i=i+f
    i1=i1+1

# Delete entries from M_resize
if(r != 0):
    pix_m = m/f - m/(f+1) # number of rows between f to f+1
    pix_n = n/f - n/(f+1) # number of columns between f to f+1

    del_rows = round(r * pix_m)
    del_cols = round(r * pix_n)

    m2 = M_resize.shape[0]
    n2 = M_resize.shape[1]
    r1 = round(m2/del_rows)
    r2 = round(n2/del_cols)

    i=0
    count=0
    while(i < m2 and count < del_rows):
        if(i % r1 == 0):
            M_resize = np.delete(M_resize, np.s_[(i-count):(i+1-count)], axis=0)
            count = count+1
        i = i+1

    i=0
    count=0
    while(i < n2 and count < del_cols):
        if(i % r2 == 0):
            M_resize = np.delete(M_resize, np.s_[(i-count):(i+1-count)], axis=1)
            count = count+1
        i = i+1


max3 = M_resize.max()
min3 = M_resize.min()
diff3 = max3 - min3

for i in range (len(M_resize)):
    for j in range(len(M_resize[0])):
        M_resize[i][j]=(((M_resize[i][j]-(min3))*(255/diff3)))

img_approx = Impil.fromarray(M_resize)
img_approx = img_approx.convert("L")
expr = str(list(uploaded)[0])
img_approx.save(expr[:expr.index('.')]+'_k1_to_k2_PCs.jpg')
    #img_approx.show()
display(img_approx)

In [ ]:
#@title Distorted Image
import random
m = M.shape[0]
n = M.shape[1]
m1 = int(0.5*m)
n1 = int(0.5*n)
del_rows_list = sorted(random.sample(range(0, m), m1))
del_cols_list = sorted(random.sample(range(0, n), n1))
M_random = M.copy()

# Delete entries from M_random
i=0
count=0
while(i < m and count < len(del_rows_list)):
    if(i == del_rows_list[count]):
        M_random = np.delete(M_random, np.s_[(i-count):(i+1-count)], axis=0)
        count = count+1
    i = i+1

i=0
count=0
while(i < n and count < len(del_cols_list)):
    if(i == del_cols_list[count]):
        M_random = np.delete(M_random, np.s_[(i-count):(i+1-count)], axis=1)
        count = count+1
    i = i+1


max3 = M_random.max()
min3 = M_random.min()
diff3 = max3 - min3

for i in range (len(M_random)):
    for j in range(len(M_random[0])):
        M_random[i][j]=(((M_random[i][j]-(min3))*(255/diff3)))

img_approx = Impil.fromarray(M_random)
img_approx = img_approx.convert("L")
expr = str(list(uploaded)[0])
img_approx.save(expr[:expr.index('.')]+'_k1_to_k2_PCs.jpg')
    #img_approx.show()
display(img_approx)

In [ ]:
#@title Image (Negative)

m = M.shape[0]
n = M.shape[1]

M_resize = np.zeros((m,n))

for i in range(m):
    for j in range(n):
        M_resize[i][j]=-1*M[i][j]

max3 = M_resize.max()
min3 = M_resize.min()
diff3 = max3 - min3

for i in range (len(M_resize)):
    for j in range(len(M_resize[0])):
        M_resize[i][j]=(((M_resize[i][j]-(min3))*(255/diff3)))

img_approx = Impil.fromarray(M_resize)
img_approx = img_approx.convert("L")
expr = str(list(uploaded)[0])
img_approx.save(expr[:expr.index('.')]+'_k1_to_k2_PCs.jpg')
    #img_approx.show()
display(img_approx)